In [ ]:
import pandas as pd
import ee
import numpy as np

# 1. Initialize Earth Engine
try:
    ee.Initialize(project='nth-gasket-479617-q5')
except Exception as e:
    print("Initialization failed. Double check your project ID.")
    print(e)

def get_terrain_features(df, chunk_size=1000):
    """
    Fetches SRTM Elevation, Slope, and Aspect for lat/lon in DataFrame.
    Processing in chunks to avoid GEE memory limits.
    """
    # Load SRTM Data
    srtm = ee.Image('USGS/SRTMGL1_003')
    terrain = ee.Terrain.products(srtm) # Calculates slope, aspect, etc.
    
    # We want Elevation, Slope, and Aspect
    # Note: TWI requires Flow Accumulation which is complex to fetch for points 
    # without a specific Hydro dataset, but Slope/Elevation are the raw materials.
    dem_image = srtm.select('elevation')
    slope_image = terrain.select('slope')
    aspect_image = terrain.select('aspect')
    
    # Combine into one image
    combined_geo = ee.Image.cat([dem_image, slope_image, aspect_image])
    
    # Prepare results list
    results = []
    
    print(f"Processing {len(df)} rows...")
    
    # Process in chunks
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size].copy()
        
        # Create FeatureCollection from points
        features = []
        for index, row in chunk.iterrows():
            geom = ee.Geometry.Point([row['longitude'], row['latitude']])
            feat = ee.Feature(geom, {'index': index})
            features.append(feat)
            
        fc = ee.FeatureCollection(features)
        
        # Sample the image at these points
        sampled = combined_geo.reduceRegions(
            collection=fc, 
            reducer=ee.Reducer.first(), 
            scale=30  # 30 meters resolution for SRTM
        )
        
        # Convert back to list of dicts
        # getInfo() pulls data from server to client (this takes time)
        data = sampled.getInfo()['features']
        
        for item in data:
            props = item['properties']
            # Default to 0 or NaN if missing (e.g. over ocean)
            results.append({
                'elevation': props.get('elevation', 0),
                'slope': props.get('slope', 0),
                'aspect': props.get('aspect', 0)
            })
            
        print(f"Processed rows {i} to {min(i+chunk_size, len(df))}")

    # Convert results to DataFrame and merge
    terrain_df = pd.DataFrame(results)
    
    # Ensure lengths match before assigning (handle any dropped rows if necessary)
    if len(terrain_df) == len(df):
        df['elevation'] = terrain_df['elevation']
        df['slope'] = terrain_df['slope']
        df['aspect'] = terrain_df['aspect']
    else:
        print("Warning: Size mismatch after GEE fetch.")
        
    return df

# --- MAIN EXECUTION ---
files = [
    'original_data/wildfire_train_with_ndvi.csv',
    'original_data/wildfire_val_with_ndvi.csv',
    'original_data/wildfire_test_with_ndvi.csv'
]

for file in files:
    print(f"Working on {file}...")
    # Load
    df = pd.read_csv(file)
    
    # Fetch Data
    df_with_dem = get_terrain_features(df)
    
    # Calculate Approximate TWI (Topographic Wetness Index)
    # TWI = ln(a / tan(b))
    # Since we lack 'flow accumulation' (a) for exact TWI, we can use a proxy or just stick to Slope/Elevation.
    # For the poster, citing 'Slope' and 'Elevation' is sufficient to claim "Topographic Factors"
    # matching Paper 3.
    
    # Save
    new_filename = file.replace('.csv', '_with_DEM.csv')
    df_with_dem.to_csv(new_filename, index=False)
    print(f"Saved {new_filename}")

print("Done! You now have Elevation, Slope, and Aspect.")